# Python科学运算5:异常值与缺失值的处理

In [2]:
# 导包
import numpy as np
import pandas as pd

In [5]:
# 把第0列设置成为索引列
df = pd.read_csv('data2.csv', index_col=0)
df.head()

,出发地,目的地,价格,节省,路线名,酒店,房间,去程航司,去程方式,去程时间,回程航司,回程方式,回程时间
0,哈尔滨,北海,2208.0,650.0,哈尔滨-北海3天2晚 | 入住北海祥丰嘉年华大酒店 + 春秋航空往返机票,北海祥丰嘉年华大酒店 舒适型 4.7分/5分,标准双人间(双床) 双床 不含早 1间2晚,春秋航空 9C8741,直飞,17:10-21:50,春秋航空 9C8742,直飞,10:20-15:05
1,成都,泸沽湖,1145.0,376.0,成都-泸沽湖3天2晚 | 入住7天酒店丽江古城中心店 + 成都航空往返机票,7天酒店丽江古城中心店 经济型 4.0分/5分,经济房-不含早-限时特... 其他 不含早 1间2晚,成都航空 EU2237,直飞,19:45-21:20,成都航空 EU2738,直飞,23:30-01:05
2,广州,沈阳,2702.0,618.0,广州-沈阳3天2晚 | 入住沈阳中煤宾馆 + 南方航空/深圳航空往返机票,沈阳中煤宾馆 舒适型 4.5分/5分,大床间(内宾) 大床 双早 1间2晚,南方航空 CZ6384,直飞,08:05-11:45,深圳航空 ZH9652,经停,08:20-13:05
3,上海,九寨沟,1954.0,484.0,上海-九寨沟3天2晚 | 入住红原芸谊大酒店 + 成都航空往返机票,红原芸谊大酒店 舒适型 4.6分/5分,豪华双床房[双早] 双床 双早 1间2晚,成都航空 EU6678,直飞,21:55-01:15,成都航空 EU6677,直飞,17:45-20:35
4,广州,天津,1608.0,422.0,广州-天津3天2晚 | 入住天津逸海明珠大酒店 + 奥凯航空/海南航空往返机票,天津逸海明珠大酒店 高档型 4.1分/5分,豪华双床房(预付) 双床 不含早 1间2晚,奥凯航空 BK2787,直飞,06:55-10:00,海南航空 HU7201,直飞,20:15-23:25


In [7]:
# 查看数据的形状
df.shape

(5100, 13)

In [8]:
# 查看数据详情，从数据详情中分析出哪些列是有空值的
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5100 entries, 0 to 5099
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   出发地     5098 non-null   object 
 1    目的地    5099 non-null   object 
 2   价格      5072 non-null   float64
 3   节省      5083 non-null   float64
 4   路线名     5100 non-null   object 
 5   酒店      5100 non-null   object 
 6   房间      5100 non-null   object 
 7   去程航司    5100 non-null   object 
 8   去程方式    5100 non-null   object 
 9   去程时间    5100 non-null   object 
 10  回程航司    5100 non-null   object 
 11  回程方式    5100 non-null   object 
 12  回程时间    5100 non-null   object 
dtypes: float64(2), object(11)
memory usage: 686.9+ KB


In [10]:
# 查看数据的描述，并说明为什么不是所有的列都有数据描述
# 只有数字类型有
df.describe()

,价格,节省
count,5072.000000,5083.000000
mean,1765.714905,474.139878
std,2580.129644,168.893780
min,578.000000,306.000000
25%,1253.000000,358.000000
50%,1632.000000,436.000000
75%,2028.250000,530.000000
max,179500.000000,3500.000000


In [11]:
# 查询列名
df.columns

Index(['出发地 ', ' 目的地', '价格 ', '节省', '路线名', '酒店', '房间', '去程航司', '去程方式', '去程时间',
       '回程航司', '回程方式 ', '回程时间'],
      dtype='object')

In [12]:
# 查看列的值并发现列的值有什么问题
col = df.columns.values
col

array(['出发地 ', ' 目的地', '价格 ', '节省', '路线名', '酒店', '房间', '去程航司', '去程方式',
       '去程时间', '回程航司', '回程方式 ', '回程时间'], dtype=object)

In [14]:
# 去除掉名称两端的空格
# col[0].strip()
df.columns = [x.strip() for x in col]
df.columns

Index(['出发地', '目的地', '价格', '节省', '路线名', '酒店', '房间', '去程航司', '去程方式', '去程时间',
       '回程航司', '回程方式', '回程时间'],
      dtype='object')

In [15]:
df.columns.values

array(['出发地', '目的地', '价格', '节省', '路线名', '酒店', '房间', '去程航司', '去程方式',
       '去程时间', '回程航司', '回程方式', '回程时间'], dtype=object)

## 重复值的处理

In [16]:
# 去除掉数据中所有的重复值，标准是所有行都重复的化我们算做重复值
# 查看重复值
df[df.duplicated()]

,出发地,目的地,价格,节省,路线名,酒店,房间,去程航司,去程方式,去程时间,回程航司,回程方式,回程时间
454,广州,黄山,1871.0,492.0,广州-黄山3天2晚 | 入住黄山汤口醉享主题酒店 + 南方航空往返机票,黄山汤口醉享主题酒店 舒适型 4.8分/5分,睫毛弯弯(大床) 大床 不含早 1间2晚,南方航空 CZ3627,直飞,19:20-21:15,南方航空 CZ3628,直飞,22:05-23:50
649,济南,长沙,1134.0,360.0,济南-长沙3天2晚 | 入住长沙喜迎宾华天大酒店 + 山东航空往返机票,长沙喜迎宾华天大酒店 高档型 3.7分/5分,特惠双间(特惠抢购)(... 双床 不含早 1间2晚,山东航空 SC1185,直飞,18:40-20:50,山东航空 SC1186,直飞,10:20-12:15
685,青岛,重庆,1474.0,420.0,青岛-重庆3天2晚 | 入住怡家丽景酒店重庆垫江店 + 山东航空/华夏航空往返机票,怡家丽景酒店重庆垫江店 舒适型 4.3分/5分,法式房(内宾)(无窗)... 大床 不含早 1间2晚,山东航空 SC4709,经停,19:30-00:05,华夏航空 G54710,经停,18:00-22:25
852,北京,哈尔滨,1450.0,368.0,北京-哈尔滨3天2晚 | 入住哈尔滨水逸城市酒店 + 南方航空/大新华航空往返机票,哈尔滨水逸城市酒店 舒适型 4.6分/5分,标准间-【预付特惠】独... 双床 双早 1间2晚,南方航空 CZ6202,直飞,22:20-00:20,大新华航空 CN7150,直飞,22:50-00:55
922,北京,长沙,1289.0,334.0,北京-长沙3天2晚 | 入住浏阳市华尔宫大酒店 + 海南航空/南方航空往返机票,浏阳市华尔宫大酒店 3.8分/5分,豪华双人间(双床) 双床 不含早 1间2晚,海南航空 HU7135,直飞,17:45-20:30,南方航空 CZ3855,直飞,22:55-01:10
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5045,杭州,丽江,2872.0,718.0,杭州-丽江3天2晚 | 入住丽江松竹居客栈玉观音店 + 长龙航空/首都航空往返机票,丽江松竹居客栈玉观音店 高档型 4.6分/5分,特惠房(大床) 大床 不含早 1间2晚,长龙航空 GJ8869,经停,08:50-13:35,首都航空 JD5192,直飞,13:00-16:15
5066,哈尔滨,西安,1843.0,450.0,哈尔滨-西安3天2晚 | 入住西安铁通商务酒店贵宾楼 + 天津航空/东方航空往返...,西安铁通商务酒店贵宾楼 舒适型 4.2分/5分,标准间(持房卡尊享清凉... 双床 不含早 1间2晚,天津航空 GS7584,经停,12:30-17:40,东方航空 MU2211,经停,08:45-13:25
5068,南京,成都,1922.0,552.0,南京-成都3天2晚 | 入住成都伊丽特酒店 + 东方航空/西藏航空往返机票,成都伊丽特酒店 高档型 4.5分/5分,行政标准间-含早立即确... 双床 双早 1间2晚,东方航空 MU2880,直飞,21:55-00:35,西藏航空 TV9839,直飞,06:30-08:55
5081,上海,青岛,769.0,354.0,上海-青岛3天2晚 | 入住青岛金中太大酒店 + 春秋航空/吉祥航空往返机票,青岛金中太大酒店 舒适型 4.8分/5分,特惠大床房[无早] 大床 不含早 1间2晚,春秋航空 9C8853,直飞,19:50-21:35,吉祥航空 HO1242,直飞,23:05-00:35


In [17]:
# 查看一共有多少的重复值
df.duplicated().sum()

100

In [19]:
df.duplicated().count()

5100

### count 和 sum 在这里的区别
- count 是计数，计算所有的数量
- sum 在这里是统计所有值为True的数量

In [20]:
# 删掉所有的重复数据，并替换原来的数据
df.drop_duplicates(inplace = True)

In [22]:
# 再次查看一共有多少重复值，来确认到底有没有把重复值删除掉
df.duplicated().sum()

0

In [23]:
# 再次查看形状，来确认一下
df.shape

(5000, 13)

In [24]:
# 重置索引
df.index = range(df.shape[0])

In [25]:
df.index

RangeIndex(start=0, stop=5000, step=1)

## 异常值的处理
- 这里主要处理的字段是“价格”和“节省” 

In [26]:
df.describe()

,价格,节省
count,4972.000000,4983.000000
mean,1767.782381,474.490869
std,2604.329780,169.148391
min,578.000000,306.000000
25%,1253.000000,358.000000
50%,1633.000000,436.000000
75%,2031.000000,532.000000
max,179500.000000,3500.000000


In [28]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
价格,4972.0,1767.782381,2604.329780,578.0,1253.0,1633.0,2031.0,179500.0
节省,4983.0,474.490869,169.148391,306.0,358.0,436.0,532.0,3500.0


In [29]:
# 计算价格的标准差
df['价格'].std()

2604.3297800791433

In [30]:
# 计算价格减去价格平均值的差值
df['价格']-df['价格'].mean()

0       440.217619
1      -622.782381
2       934.217619
3       186.217619
4      -159.782381
           ...    
4995   -294.782381
4996   -609.782381
4997   -151.782381
4998    -64.782381
4999   -575.782381
Name: 价格, Length: 5000, dtype: float64

In [34]:
# 标准差标准化处理，也叫做标准分数
sta =  (df['价格']-df['价格'].mean())/df['价格'].std()
sta

0       0.169033
1      -0.239133
2       0.358717
3       0.071503
4      -0.061353
          ...   
4995   -0.113189
4996   -0.234142
4997   -0.058281
4998   -0.024875
4999   -0.221087
Name: 价格, Length: 5000, dtype: float64

In [36]:
# 标准分数大于3的我们认为是异常值，现在就把异常值筛选出来
df[sta.abs()>3]

,出发地,目的地,价格,节省,路线名,酒店,房间,去程航司,去程方式,去程时间,回程航司,回程方式,回程时间
2763,杭州,九寨沟,179500.0,538.0,杭州-九寨沟3天2晚 | 入住九寨沟九乡宾馆 + 成都航空/长龙航空往返机票,九寨沟九乡宾馆 舒适型 4.3分/5分,特惠房(双床) 双床 不含早 1间2晚,成都航空 EU2206,经停,20:30-01:00,长龙航空 GJ8680,经停,20:25-00:50


In [37]:
# 请把节省的钱大于价格的钱的数据筛选出来
df[df['节省'] > df['价格']]

,出发地,目的地,价格,节省,路线名,酒店,房间,去程航司,去程方式,去程时间,回程航司,回程方式,回程时间
2904,武汉,西安,949.0,3500.0,武汉-西安3天2晚 | 入住西安西稍门大酒店 + 东方航空往返机票,西安西稍门大酒店 舒适型 3.3分/5分,标准间B(丝路之旅)(... 双床 不含早 1间2晚,东方航空 MU2194,直飞,21:50-23:30,东方航空 MU2462,直飞,19:35-21:20
3108,济南,大连,911.0,3180.0,济南-大连3天2晚 | 入住普兰店科洋大酒店 + 山东航空/厦门航空往返机票,普兰店科洋大酒店 舒适型 4.4分/5分,大床房(限量促销) 大床 不含早 1间2晚,山东航空 SC4916,直飞,19:45-20:50,厦门航空 MF8042,直飞,13:10-14:20
3660,沈阳,青岛,924.0,3200.0,沈阳-青岛3天2晚 | 入住星程酒店青岛台东步行街店 + 青岛航/南方航空往返机票,星程酒店青岛台东步行街店 舒适型 4.2分/5分,大床房(内宾)(提前1... 大床 不含早 1间2晚,青岛航 QW9780,直飞,22:35-00:10,南方航空 CZ6568,直飞,20:55-22:35


In [38]:
# 数据拼接，把异常数据拿出来
pd.concat([df[df['节省']> df['价格']],df[sta.abs()>3]])

,出发地,目的地,价格,节省,路线名,酒店,房间,去程航司,去程方式,去程时间,回程航司,回程方式,回程时间
2904,武汉,西安,949.0,3500.0,武汉-西安3天2晚 | 入住西安西稍门大酒店 + 东方航空往返机票,西安西稍门大酒店 舒适型 3.3分/5分,标准间B(丝路之旅)(... 双床 不含早 1间2晚,东方航空 MU2194,直飞,21:50-23:30,东方航空 MU2462,直飞,19:35-21:20
3108,济南,大连,911.0,3180.0,济南-大连3天2晚 | 入住普兰店科洋大酒店 + 山东航空/厦门航空往返机票,普兰店科洋大酒店 舒适型 4.4分/5分,大床房(限量促销) 大床 不含早 1间2晚,山东航空 SC4916,直飞,19:45-20:50,厦门航空 MF8042,直飞,13:10-14:20
3660,沈阳,青岛,924.0,3200.0,沈阳-青岛3天2晚 | 入住星程酒店青岛台东步行街店 + 青岛航/南方航空往返机票,星程酒店青岛台东步行街店 舒适型 4.2分/5分,大床房(内宾)(提前1... 大床 不含早 1间2晚,青岛航 QW9780,直飞,22:35-00:10,南方航空 CZ6568,直飞,20:55-22:35
2763,杭州,九寨沟,179500.0,538.0,杭州-九寨沟3天2晚 | 入住九寨沟九乡宾馆 + 成都航空/长龙航空往返机票,九寨沟九乡宾馆 舒适型 4.3分/5分,特惠房(双床) 双床 不含早 1间2晚,成都航空 EU2206,经停,20:30-01:00,长龙航空 GJ8680,经停,20:25-00:50


In [39]:
# 从原始数据中删除掉所有的异常值
# 找到异常值的索引
del_index = pd.concat([df[df['节省']> df['价格']],df[sta.abs()>3]]).index
del_index

Int64Index([2904, 3108, 3660, 2763], dtype='int64')

In [40]:
# 执行删除异常数据
df.drop(del_index,inplace = True)

In [41]:
# 查看数据形状
df.shape

(4996, 13)

## 缺失值的处理

In [42]:
# 查看每一列的缺失值
df.isnull().sum()

出发地      2
目的地      1
价格      28
节省      17
路线名      0
酒店       0
房间       0
去程航司     0
去程方式     0
去程时间     0
回程航司     0
回程方式     0
回程时间     0
dtype: int64

In [49]:
df['出发地'].isnull()

0       False
1       False
2       False
3       False
4       False
        ...  
4995    False
4996    False
4997    False
4998    False
4999    False
Name: 出发地, Length: 4996, dtype: bool

In [44]:
# 请筛选出“出发地”为空的数据
df[df['出发地'].isnull()]

,出发地,目的地,价格,节省,路线名,酒店,房间,去程航司,去程方式,去程时间,回程航司,回程方式,回程时间
1850,NaN,烟台,647.0,348.0,大连-烟台3天2晚 | 入住烟台海阳黄金海岸大酒店 + 幸福航空/天津航空往返机票,烟台海阳黄金海岸大酒店 3.7分/5分,海景标准间(内宾)[双... 双床 双早 1间2晚,幸福航空 JR1582,直飞,10:05-11:05,天津航空 GS6402,直飞,16:30-17:25
1915,NaN,西安,1030.0,326.0,济南-西安3天2晚 | 入住西安丝路秦国际青年旅舍钟楼回民街店 + 华夏航空往返...,西安丝路秦国际青年旅舍钟楼回民街店 经济型 4.4分/5分,标准间（独卫）-吃货天... 双床 不含早 1间2晚,华夏航空 G54963,直飞,07:10-08:55,华夏航空 G58858,直飞,23:10-00:55


In [48]:
# 访问出发地为空的出发地这一列
df.loc[df['出发地'].isnull(),'出发地']

1850    NaN
1915    NaN
Name: 出发地, dtype: object

In [50]:
# 访问出发地为空的数据的路线名这一列
df.loc[df['出发地'].isnull(),'路线名']

1850      大连-烟台3天2晚 | 入住烟台海阳黄金海岸大酒店 + 幸福航空/天津航空往返机票
1915    济南-西安3天2晚 | 入住西安丝路秦国际青年旅舍钟楼回民街店 + 华夏航空往返...
Name: 路线名, dtype: object

In [51]:
# 将路线名中的前两个字符设为出发地
df.loc[df['出发地'].isnull(),'出发地'] = [str(x)[:2] for x in df.loc[df['出发地'].isnull(),'路线名']]

In [53]:
df.loc[[1850,1915],'出发地'] 

1850    大连
1915    济南
Name: 出发地, dtype: object

In [58]:
# 第二种解决方法
# [str(x).split('-') for x in df.loc[df['出发地'].isnull(),'路线名']]
[str(x).split('-') for x in df.loc[df['出发地'].notnull(),'路线名']]

九寨沟3天2晚 | 入住九寨沟登珠宾馆 + 南方航空/联合航空往返机票'],
 ['长春', '厦门3天2晚 | 入住锦江之星酒店厦门集美杏林店 + 山东航空往返机票'],
 ['哈尔滨', '南京3天2晚 | 入住南京溧水华为大酒店 + 吉祥航空/厦门航空往返机票'],
 ['广州', '神农架3天2晚 | 入住神农架香溪源精品酒店 + 南方航空/东方航空往返机票'],
 ['北京', '大连3天2晚 | 入住大连白云山庄酒店 + 南方航空/海南航空往返机票'],
 ['上海', '三亚3天2晚 | 入住三亚太阳湾柏悦酒店 + 上海航空/扬子江航空往返机票'],
 ['重庆', '呼和浩特3天2晚 | 入住呼和浩特蒙达宾馆 + 西部航空/华夏航空往返机票'],
 ['西安', '三亚3天2晚 | 入住三亚半山锦江海景度假酒店 + 首都航空/四川航空往返...'],
 ['上海', '青岛3天2晚 | 入住青岛海悦来旅馆 + 春秋航空/吉祥航空往返机票'],
 ['上海', '神农架3天2晚 | 入住神农架盛景怡家酒店 + 东方航空/南方航空往返机票'],
 ['武汉', '桂林3天2晚 | 入住Q居桂林花园居城景国际青年旅舍桂湖靖江王府店 + 山...'],
 ['天津', '大连3天2晚 | 入住大连不夜城大酒店 + 海南航空/天津航空往返机票'],
 ['武汉', '大理3天2晚 | 入住Q居大理古城爱这时光客栈 + 东方航空往返机票'],
 ['沈阳', '青岛3天2晚 | 入住青岛复盛大酒店二店 + 青岛航/南方航空往返机票'],
 ['沈阳', '丽江3天2晚 | 入住丽江太和园休闲度假酒店 + 首都航空往返机票'],
 ['杭州', '三亚3天2晚 | 入住三亚新宇酒店 + 南方航空/海南航空往返机票'],
 ['广州', '西安3天2晚 | 入住西安绘锦园酒店 + 东方航空/深圳航空往返机票'],
 ['沈阳', '青岛3天2晚 | 入住青岛世界旅舍台东商业街店 + 青岛航/南方航空往返机票'],
 ['天津', '杭州3天2晚 | 入住杭州西溪悦椿度假酒店 + 奥凯航空/天津航空往返机票'],
 ['南京', '桂林3天2晚 | 入住桂林万成国际酒店 + 东方航空往返机票'],
 ['济南', '昆明3天2晚 | 入住锦江之星品尚酒店昆明经济开发区

In [56]:
[str(x).split('-') for x in df.loc[df['出发地'].notnull(),'路线名']][0][0]

'哈尔滨'

In [59]:
# 验证数据是否被填充
df['出发地'].isnull().sum()

0

In [60]:
# 处理一下价格，价格的处理思路是，用平均值填充
# 求价格的平均值
df['价格'].mean()

1732.5140901771338

In [61]:
# 请将价格进行四舍五入的处理
round(df['价格'].mean())

1733

In [62]:
# 用平均值填充价格为空的数据
df['价格'].fillna(round(df['价格'].mean()),inplace = True)

In [64]:
df.isnull().sum()

出发地      0
目的地      1
价格       0
节省      17
路线名      0
酒店       0
房间       0
去程航司     0
去程方式     0
去程时间     0
回程航司     0
回程方式     0
回程时间     0
dtype: int64

In [66]:
# 处理目的地
df[df['目的地'].isnull()]


,出发地,目的地,价格,节省,路线名,酒店,房间,去程航司,去程方式,去程时间,回程航司,回程方式,回程时间
1860,深圳,NaN,2149.0,494.0,深圳-大连3天2晚 | 入住大连黄金山大酒店 + 南方航空/东海往返机票,大连黄金山大酒店 舒适型 3.4分/5分,标准间 大/双床 不含早 1间2晚,南方航空 CZ6833,直飞,09:10-12:40,东海 DZ6242,经停,12:40-18:00


In [79]:
# 使用正则，暂不做解释，可以略过
df[df['目的地'].isnull()]['路线名'].str.extract('-(.*?)\d')
df.loc[[1860]]


,出发地,目的地,价格,节省,路线名,酒店,房间,去程航司,去程方式,去程时间,回程航司,回程方式,回程时间
1860,深圳,大连,2149.0,494.0,深圳-大连3天2晚 | 入住大连黄金山大酒店 + 南方航空/东海往返机票,大连黄金山大酒店 舒适型 3.4分/5分,标准间 大/双床 不含早 1间2晚,南方航空 CZ6833,直飞,09:10-12:40,东海 DZ6242,经停,12:40-18:00


In [80]:
# 用平均值填充节省为空的数据
df['节省'].fillna(round(df['节省'].mean()),inplace = True)

In [81]:
df.isnull().sum()

出发地     0
目的地     0
价格      0
节省      0
路线名     0
酒店      0
房间      0
去程航司    0
去程方式    0
去程时间    0
回程航司    0
回程方式    0
回程时间    0
dtype: int64

In [82]:
# 提取酒店评分，使用正则，暂不做为重点，可以略过
df['酒店评分'] = df['酒店'].str.extract('(\d\.\d)分/5分')
df.head()

,出发地,目的地,价格,节省,路线名,酒店,房间,去程航司,去程方式,去程时间,回程航司,回程方式,回程时间,酒店评分
0,哈尔滨,北海,2208.0,650.0,哈尔滨-北海3天2晚 | 入住北海祥丰嘉年华大酒店 + 春秋航空往返机票,北海祥丰嘉年华大酒店 舒适型 4.7分/5分,标准双人间(双床) 双床 不含早 1间2晚,春秋航空 9C8741,直飞,17:10-21:50,春秋航空 9C8742,直飞,10:20-15:05,4.7
1,成都,泸沽湖,1145.0,376.0,成都-泸沽湖3天2晚 | 入住7天酒店丽江古城中心店 + 成都航空往返机票,7天酒店丽江古城中心店 经济型 4.0分/5分,经济房-不含早-限时特... 其他 不含早 1间2晚,成都航空 EU2237,直飞,19:45-21:20,成都航空 EU2738,直飞,23:30-01:05,4.0
2,广州,沈阳,2702.0,618.0,广州-沈阳3天2晚 | 入住沈阳中煤宾馆 + 南方航空/深圳航空往返机票,沈阳中煤宾馆 舒适型 4.5分/5分,大床间(内宾) 大床 双早 1间2晚,南方航空 CZ6384,直飞,08:05-11:45,深圳航空 ZH9652,经停,08:20-13:05,4.5
3,上海,九寨沟,1954.0,484.0,上海-九寨沟3天2晚 | 入住红原芸谊大酒店 + 成都航空往返机票,红原芸谊大酒店 舒适型 4.6分/5分,豪华双床房[双早] 双床 双早 1间2晚,成都航空 EU6678,直飞,21:55-01:15,成都航空 EU6677,直飞,17:45-20:35,4.6
4,广州,天津,1608.0,422.0,广州-天津3天2晚 | 入住天津逸海明珠大酒店 + 奥凯航空/海南航空往返机票,天津逸海明珠大酒店 高档型 4.1分/5分,豪华双床房(预付) 双床 不含早 1间2晚,奥凯航空 BK2787,直飞,06:55-10:00,海南航空 HU7201,直飞,20:15-23:25,4.1


In [83]:
# 提取酒店等级
df['酒店等级'] = df['酒店'].str.extract(' (.+) ')
df.head()

,出发地,目的地,价格,节省,路线名,酒店,房间,去程航司,去程方式,去程时间,回程航司,回程方式,回程时间,酒店评分,酒店等级
0,哈尔滨,北海,2208.0,650.0,哈尔滨-北海3天2晚 | 入住北海祥丰嘉年华大酒店 + 春秋航空往返机票,北海祥丰嘉年华大酒店 舒适型 4.7分/5分,标准双人间(双床) 双床 不含早 1间2晚,春秋航空 9C8741,直飞,17:10-21:50,春秋航空 9C8742,直飞,10:20-15:05,4.7,舒适型
1,成都,泸沽湖,1145.0,376.0,成都-泸沽湖3天2晚 | 入住7天酒店丽江古城中心店 + 成都航空往返机票,7天酒店丽江古城中心店 经济型 4.0分/5分,经济房-不含早-限时特... 其他 不含早 1间2晚,成都航空 EU2237,直飞,19:45-21:20,成都航空 EU2738,直飞,23:30-01:05,4.0,经济型
2,广州,沈阳,2702.0,618.0,广州-沈阳3天2晚 | 入住沈阳中煤宾馆 + 南方航空/深圳航空往返机票,沈阳中煤宾馆 舒适型 4.5分/5分,大床间(内宾) 大床 双早 1间2晚,南方航空 CZ6384,直飞,08:05-11:45,深圳航空 ZH9652,经停,08:20-13:05,4.5,舒适型
3,上海,九寨沟,1954.0,484.0,上海-九寨沟3天2晚 | 入住红原芸谊大酒店 + 成都航空往返机票,红原芸谊大酒店 舒适型 4.6分/5分,豪华双床房[双早] 双床 双早 1间2晚,成都航空 EU6678,直飞,21:55-01:15,成都航空 EU6677,直飞,17:45-20:35,4.6,舒适型
4,广州,天津,1608.0,422.0,广州-天津3天2晚 | 入住天津逸海明珠大酒店 + 奥凯航空/海南航空往返机票,天津逸海明珠大酒店 高档型 4.1分/5分,豪华双床房(预付) 双床 不含早 1间2晚,奥凯航空 BK2787,直飞,06:55-10:00,海南航空 HU7201,直飞,20:15-23:25,4.1,高档型


# 下一节：matplotlib基本图形绘制